In [87]:
#Aug 30, 2022

import pandas as pd
import numpy as np

In [88]:
Drug_ID_map = pd.read_csv("/Users/guangrong/Documents/GitHub_project/drug_response_kp/Data/GDSC/screened_compunds_rel_8.2 2_withID.csv")


In [89]:
Drug_ID_map
dic_Drug_name_id = {}
for i in range(0,Drug_ID_map.shape[0]):
    DrugName = Drug_ID_map.iloc[i,2]
    ID = Drug_ID_map.iloc[i,7]
    if (ID != np.nan ) and type(ID) == str:
        dic_Drug_name_id[DrugName] = ID

In [90]:
len(dic_Drug_name_id)

416

In [91]:
input_dir = "/Users/guangrong/Documents/GitHub_project/fastqpi_BigGIM/KGs/TCGA_mut/"

In [92]:
#generating a ID mapping dictionary for disease name and mondo ID
TCGA_Mondo_map = pd.read_csv(input_dir + "Mondo_TCGA.csv")
TCGA_Mondo_map
dic_mondo_map = {}
for i in range(0, TCGA_Mondo_map.shape[0]):
    dic_mondo_map['TCGA_' + list(TCGA_Mondo_map['Study Abbreviation'])[i]] = list(TCGA_Mondo_map['MONDO'])[i]
    dic_mondo_map['TCGA-' + list(TCGA_Mondo_map['Study Abbreviation'])[i]] = list(TCGA_Mondo_map['MONDO'])[i]


In [93]:
#Generate an ID mapping dictionary for gene symbol and NCBI gene ID
ID_map = pd.read_csv("/Users/guangrong/Documents/GitHub_project/fastqpi_BigGIM/KGs/ID_convert/Table_ID_convert_9.19.2021.tsv", sep = '\t')
dic_ID_symbol2ncbi ={}
dic_ID_ncbi2symbol = {}
import math
ID_map.astype(str)
for i in range(0,ID_map.shape[0]):
    symbol = ID_map['Approved_symbol'][i]
    
    ncbi_id = ID_map['NCBI_Gene_ID'][i]
    ensemble_id = ID_map['Ensembl_gene_ID'][i]
    if math.isnan(ncbi_id) == False :
        ncbi_id = int(ncbi_id)
        if symbol not in dic_ID_symbol2ncbi:
            dic_ID_symbol2ncbi[symbol] = ncbi_id
        else:
            print(symbol)
for i in range(0,ID_map.shape[0]):
    alias_list = str(ID_map["Alias_symbols"][i]).split(',')
    ncbi_id = ID_map['NCBI_Gene_ID'][i]
    ensemble_id = ID_map['Ensembl_gene_ID'][i]
    if len(alias_list) > 0:
        for alias in alias_list:
            if alias not in dic_ID_symbol2ncbi:
                dic_ID_symbol2ncbi[alias] = ncbi_id
dic_ID_symbol2ncbi['MEN1'] = 4221
dic_ID_symbol2ncbi['TSC1']= 7248
dic_ID_symbol2ncbi['WHSC1'] = 7468
dic_ID_symbol2ncbi['MET'] = 4233
dic_ID_symbol2ncbi['HIST1H1E'] = 3008
dic_ID_symbol2ncbi['FLNA'] = 2316
dic_ID_symbol2ncbi["FAM46D"] = 169966
dic_ID_symbol2ncbi["HIST1H1C"] = 3006

In [94]:
cellLines = pd.read_excel("/Users/guangrong/Documents/GitHub_project/drug_response_kp/Data/GDSC/Cell_Lines_Details.xlsx", sheet_name= "Cell line details")

/Users/guangrong/.pyenv/versions/3.10.0/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [161]:
set(cellLines['Cancer Type TCGA label'])

{'ACC',
 'ALL',
 'BLCA',
 'BRCA',
 'CESC',
 'CLL',
 'COAD/READ',
 'DLBC',
 'ESCA',
 'GBM',
 'HNSC',
 'KIRC',
 'LAML',
 'LCML',
 'LGG',
 'LIHC',
 'LUAD',
 'LUSC',
 'MB',
 'MESO',
 'MM',
 'NB',
 'OV',
 'PAAD',
 'PRAD',
 'SCLC',
 'SKCM',
 'STAD',
 'THCA',
 'UCEC',
 'UNABLE TO CLASSIFY',
 nan}

In [95]:
drug_response = pd.read_excel("/Users/guangrong/Documents/GitHub_project/drug_response_kp/Data/GDSC/GDSC1_fitted_dose_response_25Feb20.xlsx")

In [373]:

tumor_type = "BLCA"
tumor_type = "BRCA"
tumor_type = "CESC"
tumor_type = "DLBC"
tumor_type = "ESCA"
tumor_type = "GBM"
tumor_type = "HNSC"
tumor_type = "KIRC"
tumor_type = "LAML"
tumor_type = "LCML"
tumor_type = "LGG"
tumor_type = "LIHC"
tumor_type = "LUAD"
tumor_type = "LUSC"
tumor_type = "MESO"
tumor_type = "OV"
tumor_type = "PAAD"
tumor_type = "PRAD"
tumor_type = "SKCM"
tumor_type = "STAD"
tumor_type ="THCA"
tumor_type = "UCEC"

In [374]:
tumor_type_cellLines = list(cellLines.loc[cellLines['Cancer Type TCGA label'] == tumor_type]['COSMIC identifier'])
tumor_specific_drugResponse = drug_response.loc[drug_response['COSMIC_ID'].isin(tumor_type_cellLines)]

In [375]:
sele_drugs = []
for drug in set(tumor_specific_drugResponse['DRUG_NAME']):
    if tumor_specific_drugResponse.loc[tumor_specific_drugResponse['DRUG_NAME'] == drug].shape[0] >10:
        drug_response_curr = tumor_specific_drugResponse.loc[tumor_specific_drugResponse['DRUG_NAME'] == drug]
        if drug_response_curr.loc[drug_response_curr['LN_IC50'] < 0].shape[0] > 5:
            sele_drugs.append(drug)
            

In [376]:
#sele_drugs
DrugResponse_mut = pd.read_csv("/Users/guangrong/Documents/GitHub_project/fastqpi_BigGIM/KGs/GeneMutation_based_drug_sensitivity/"+tumor_type+"_mut_IC50_GDSC.csv")
DrugResponse_mut_sele = DrugResponse_mut.loc[DrugResponse_mut['Name'].isin(sele_drugs)]
DrugResponse_mut_sele = DrugResponse_mut_sele.loc[DrugResponse_mut_sele['pvalue'] < 0.05]

In [377]:
subject_name_list = DrugResponse_mut_sele['Gene']
object_name_list = DrugResponse_mut_sele['Name']

subject_name_list_new = []
object_name_list_new = []

subject_id_list = []
object_id_list = []

object_id_prefix = []

predicate_list = []

supporting_study_size = []

P_value_list = []

for i in range(0, DrugResponse_mut_sele.shape[0]):
    gene = DrugResponse_mut_sele.iloc[i,1]
    drug = DrugResponse_mut_sele.iloc[i,4]
    es = DrugResponse_mut_sele.iloc[i,7]
    mut_samples = DrugResponse_mut_sele.iloc[i,9]
    wt_samples = DrugResponse_mut_sele.iloc[i,10]
    P = DrugResponse_mut_sele.iloc[i,5]
    
    if (gene in dic_ID_symbol2ncbi) and (drug in dic_Drug_name_id) and dic_Drug_name_id[drug].startswith("PUBCHEM"):
        subject_name_list_new.append(gene)
        subject_id_list.append(dic_ID_symbol2ncbi[gene])
        object_name_list_new.append(drug)
        object_id_list.append(dic_Drug_name_id[drug].split(":")[1])
        object_id_prefix.append(dic_Drug_name_id[drug].split(":")[0])
        
        if es > 0:
            predicate_list.append("associated_with_resistance_to")
        else:
            predicate_list.append("associated_with_sensitivity_to")
        
        
                
        supporting_study_size.append(mut_samples+wt_samples)
        
        P_value_list.append(P)


In [378]:
Result = pd.DataFrame()
Result['subject_id'] = subject_id_list
Result['object_id'] = object_id_list
Result['subject_id_prefix'] = ['NCBIGene'] * len(subject_id_list)
Result['object_id_prefix'] = object_id_prefix
Result['subject_name'] = subject_name_list_new
Result['object_name'] = object_name_list_new
Result['predicate'] = predicate_list
Result['Data_set'] = ['GDSC'] * len(subject_id_list)
Result['publications'] = ["PMID:27397505"]*len(subject_id_list)
Result['subject_category'] = ['Gene']*len(subject_id_list)
Result['object_category'] = ['SmallMolecule']* len(subject_id_list)
Result['statistics_method'] = ['T_test'] * len(object_id_list)
Result['supporting_study_size'] = supporting_study_size
Result['P_value'] = P_value_list
Result['subject_aspect_qualifier'] = "Genetic variants"
Result['context_qualifier'] = [dic_mondo_map["TCGA-"+tumor_type]]*len(subject_id_list)
Result['subject_id'] = Result['subject_id'].astype(int)

In [379]:
Result.to_csv("../Input_Table/DrugResponse_mut_"+tumor_type+".csv", index = None)